# Code for Cahn-Hilliard phase separation with mechanical coupling.


## 2D phase separation study.




Degrees of freedom: 
  - scalar chemical potential: we use normalized  mu = mu/RT 
  - species concentration:  we use normalized  c= Omega*cmat 
  
### Units:
- Length: um
- Mass: kg
- Time: s
- Amount of substance: pmol
- Temperature: K
- Mass density: kg/um^3
- Force: uN
- Stress: MPa
- Energy: pJ
- Species concentration: pmol/um^3
- Chemical potential: pJ/pmol
- Molar volume: um^3/pmol
- Species diffusivity: um^2/s
- Boltzmann Constant: 1.38E-11 pJ/K
- Gas constant: 8.314  pJ/(pmol K)

### By
  Eric Stewart      and      Lallit Anand
ericstew@mit.edu            anand@mit.edu

October 2023

Modified for FenicsX by Jorge Nin
jorgenin@mit.edu

In [1]:
import numpy as np


from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import fem, mesh, io, plot, log, default_scalar_type
from dolfinx.fem import Constant, dirichletbc, Function, FunctionSpace, Expression
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.io import VTXWriter

import ufl
from ufl import (
    TestFunction,
    TrialFunction,
    Identity,
    grad,
    det,
    div,
    dev,
    inv,
    tr,
    sqrt,
    conditional,
    gt,
    dx,
    inner,
    derivative,
    dot,
    ln,
    split,
    tanh,
    as_tensor,
    as_vector,
    ge
)

from hilliard_models import Cahn_Hillard_axi_symmetric
from datetime import datetime
from dolfinx.plot import vtk_mesh

import pyvista

pyvista.set_jupyter_backend("client")
## Define temporal parameters
import random
import gmsh

# DEFINE GEOMETRY

In [2]:
problemName = "Canh Hillard Mechanical Axi Symetric Oval"
a = .5 #Semi major axis
b = .3 #Semi minor axis
lc = .01
points = [ [0,0],[0,a],[b,0]]
mesh_comm = MPI.COMM_WORLD
model_rank = 0


    
if mesh_comm.rank == model_rank:
    
    gmsh.initialize()
    gmsh.model.add("oval")
    meshpoints = []
    for point in points: 
        #print(point)
        meshpoints.append( gmsh.model.geo.add_point(point[0],point[1],0,lc))
    

    l3 = gmsh.model.geo.add_ellipse_arc(meshpoints[1],meshpoints[0], meshpoints[1], meshpoints[2])
    l1 = gmsh.model.geo.add_line(meshpoints[0], meshpoints[1])
    l2 = gmsh.model.geo.add_line(meshpoints[2], meshpoints[0])
    #l3 =gmsh.model.geo.add_line(meshpoints[2], meshpoints[0])
    
    loop = gmsh.model.geo.add_curve_loop([l1, l2, l3])
    
    surface = gmsh.model.geo.addPlaneSurface([1],1)
    
    
    gdim =2
    
    gmsh.model.geo.synchronize()
    
    gmsh.model.addPhysicalGroup(2, [surface],name="My Surface")
    
    BOTTOM_TAG, LEFT_TAG, OUTSIDE_TAG = 1, 2, 3
    gmsh.model.addPhysicalGroup(1,[l2],BOTTOM_TAG,"Bottom")
    gmsh.model.addPhysicalGroup(1,[l3],OUTSIDE_TAG,"Outside")
    gmsh.model.addPhysicalGroup(1,[l1],LEFT_TAG,"Left")
    
    
    gmsh.model.mesh.generate(2)
    

    
    gmsh.write("mesh/oval.msh")
    gmsh.finalize()
    
    


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Ellipse)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Line)
Info    : Done meshing 1D (Wall 0.00132625s, CPU 0.001479s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0228706s, CPU 0.022482s)
Info    : 1460 nodes 2921 elements
Info    : Writing 'mesh/oval.msh'...
Info    : Done writing 'mesh/oval.msh'


# Simulation Time Control

In [3]:
t = 0.0  # initialization of time
Ttot = 1800  # total simulation time
dt = 0.01  # Initial time step size, here we will use adaptive time-stepping

# Create Problem

In [4]:
from dolfinx.io.gmshio import read_from_msh
domain, cell_tags, facet_tags = read_from_msh("mesh/oval.msh", MPI.COMM_WORLD, 0,gdim=2)
dk = Constant(domain,dt)

cri = 0.005
crf = 0.995
def InitFunction(hillard_problem : Cahn_Hillard_axi_symmetric):

    # Some quick definitions to init random values
    V, _ = hillard_problem.ME.sub(1).collapse()
    cBar_init = Function(V)

    cBar_init.interpolate(
        lambda x: np.full_like(x[1],cri)
    ) 

    fc_rand =  (
        ln(cBar_init / (1 - cBar_init)) + hillard_problem.chi * (1 - 2 * cBar_init)
    )  # use that relation to initate the two different sub expressions

    concentration = Expression(
        hillard_problem.Omega*hillard_problem.cMax*cri,
        hillard_problem.ME.sub(2).element.interpolation_points(),
    )
    hillard_problem.w.sub(2).interpolate(concentration)

    chemical_potential = Expression(
        fc_rand , hillard_problem.ME.sub(1).element.interpolation_points()
    )
    hillard_problem.w.sub(1).interpolate(chemical_potential)

    hillard_problem.w.sub(0).interpolate(lambda x: np.zeros((2, x.shape[1])))

    hillard_problem.w_old.x.array[:] = hillard_problem.w.x.array
    hillard_problem.w_old_2.x.array[:] = hillard_problem.w.x.array

hillard_problem = Cahn_Hillard_axi_symmetric(domain,init_func=InitFunction)
hillard_problem.Kinematics()
hillard_problem.WeakForms(dk)

Info    : Reading 'mesh/oval.msh'...
Info    : 7 entities
Info    : 1460 nodes
Info    : 2918 elements
Info    : Done reading 'mesh/oval.msh'


# Boundary Conditions

In [5]:
# Just fix the sides to make sure they don't move

u_bc = np.array((0), dtype=default_scalar_type)




left_dofs = fem.locate_dofs_topological(hillard_problem.ME.sub(0).sub(0), facet_tags.dim, facet_tags.find(LEFT_TAG)) #we don't want it to move in the x direction
bottom_dofs = fem.locate_dofs_topological(hillard_problem.ME.sub(0).sub(1), facet_tags.dim, facet_tags.find(BOTTOM_TAG)) #we don't want it to move in the y direction
bcs = [fem.dirichletbc(u_bc, left_dofs, hillard_problem.ME.sub(0).sub(0)),
       fem.dirichletbc(u_bc, bottom_dofs, hillard_problem.ME.sub(0).sub(1))]


In [7]:
from dolfinx.fem.petsc import assemble_matrix, create_matrix, create_vector
import sys
from slepc4py import SLEPc

jacobian = fem.form(hillard_problem.a)
Residual = fem.form(hillard_problem.Res)
A = create_matrix(jacobian)
L = create_vector(Residual)

assemble_matrix(A, jacobian, bcs=bcs)
A.assemble()
B = A.copy()

C = A.transposeMatMult(B)

C.setOption(PETSc.Mat.Option.SYMMETRIC, True)

C.assemble()
eigenSolver = SLEPc.EPS()
eigenSolver.create(comm=mesh_comm)

eigenSolver.setOperators(C)
eigenSolver.setProblemType(SLEPc.EPS.ProblemType.HEP)
#eigenSolver.setType(eigenSolver.Type.SCALAPACK)
eigenSolver.setWhichEigenpairs(eigenSolver.Which.SMALLEST_REAL)
eigenSolver.setTolerances(1.0e-5, 1000)
eigenSolver.setKrylovSchurRestart(0.6)
eigenSolver.setFromOptions()
eigenSolver.solve()

eigen_max = eigenSolver.getEigenvalue(0)

print(eigen_max)


eigenSolver.setWhichEigenpairs(eigenSolver.Which.LARGEST_REAL)
eigenSolver.solve()
eigen_min = eigenSolver.getEigenvalue(0)
print(eigen_min)


In [ ]:
nev, ncv, mpd = eigenSolver.getDimensions()
print("Number of requested eigenvalues: %i" % nev)

Number of requested eigenvalues: 1


In [ ]:
eigenSolver.getConverged()
arr = C[:,:]

In [ ]:
import scipy

scipy.linalg.eigvalsh(arr)

In [ ]:
value = np.linalg.cond(arr)